In [1]:
spark.sql("""SET -v""").select("key", "value").show(5, false)

Intitializing Scala interpreter ...

Spark Web UI available at http://EM2021002861.bosonit.local:4040
SparkContext available as 'sc' (version = 3.1.1, master = local[*], app id = local-1623831370539)
SparkSession available as 'spark'


+---------------------------------------------------------+----------------------------------------------------------------+
|key                                                      |value                                                           |
+---------------------------------------------------------+----------------------------------------------------------------+
|spark.sql.adaptive.advisoryPartitionSizeInBytes          |<value of spark.sql.adaptive.shuffle.targetPostShuffleInputSize>|
|spark.sql.adaptive.coalescePartitions.enabled            |true                                                            |
|spark.sql.adaptive.coalescePartitions.initialPartitionNum|<undefined>                                                     |
|spark.sql.adaptive.coalescePartitions.minPartitionNum    |<undefined>                                                     |
|spark.sql.adaptive.enabled                               |false                                                           |


In [4]:
spark.conf.get("spark.sql.shuffle.partitions")

res3: String = 200


In [5]:
spark.conf.set("spark.sql.shuffle.partitions",5)

In [6]:
spark.conf.get("spark.sql.shuffle.partitions")

res5: String = 5


In [7]:
spark.conf.set("spark.sql.shuffle.partitions",20)

In [8]:
spark.conf.get("spark.sql.shuffle.partitions")

res7: String = 20


In [9]:
val df = spark.range(1 * 10000000).toDF("id").withColumn("square", $"id" * $"id")

df: org.apache.spark.sql.DataFrame = [id: bigint, square: bigint]


Cuando un dataframe de mucho tamaño lo guardamos en la memoria cache, mejoramos su posterior uso ya que se reducira el tiempo en ejecución de los comandos

In [10]:
df.cache()

res8: df.type = [id: bigint, square: bigint]


In [11]:
df.count()

res9: Long = 10000000


In [12]:
df.count()

res10: Long = 10000000


Lo mismo vemos que pasa usando la funcion persist(), esta funcion hace lo mismo que el cache pero puede controlar donde y como se guardan las cosas

In [14]:
import org.apache.spark.storage.StorageLevel

import org.apache.spark.storage.StorageLevel


In [18]:
val df1 = spark.range(1 * 100000000).toDF("id").withColumn("square", $"id" * $"id")

df1: org.apache.spark.sql.DataFrame = [id: bigint, square: bigint]


In [19]:
df1.persist(StorageLevel.DISK_ONLY)

res15: df1.type = [id: bigint, square: bigint]


In [20]:
df1.count()

res16: Long = 100000000


In [22]:
df1.count()

res18: Long = 100000000


In [23]:
df1.unpersist()

res19: df1.type = [id: bigint, square: bigint]


Shuffle Sort Merge Join

Unir dos dataset de gran tamaño

In [2]:
import scala.util.Random

import scala.util.Random


In [3]:
spark.conf.get("spark.sql.autoBroadcastJoinThreshold")

res1: String = 10485760b


In [22]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "10485760b")

In [4]:
spark.conf.get("spark.sql.autoBroadcastJoinThreshold", "-1")

res2: String = -1


In [5]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")

In [23]:
spark.conf.get("spark.sql.autoBroadcastJoinThreshold")

res14: String = 10485760b


In [7]:
var states = scala.collection.mutable.Map[Int, String]()

states: scala.collection.mutable.Map[Int,String] = Map()


In [8]:
var items = scala.collection.mutable.Map[Int, String]()

items: scala.collection.mutable.Map[Int,String] = Map()


In [9]:
val rnd = new scala.util.Random(42)

rnd: scala.util.Random = scala.util.Random@587a3024


In [10]:
states += (0 -> "AZ", 1 -> "CO", 2-> "CA", 3-> "TX", 4 -> "NY", 5-> "MI")

res5: scala.collection.mutable.Map[Int,String] = Map(2 -> CA, 5 -> MI, 4 -> NY, 1 -> CO, 3 -> TX, 0 -> AZ)


In [12]:
items += (0 -> "SKU-0", 1 -> "SKU-1", 2-> "SKU-2", 3-> "SKU-3", 4 -> "SKU-4", 5-> "SKU-5")

res6: scala.collection.mutable.Map[Int,String] = Map(2 -> SKU-2, 5 -> SKU-5, 4 -> SKU-4, 1 -> SKU-1, 3 -> SKU-3, 0 -> SKU-0)


In [13]:
val usersDF = (0 to 1000000).map(id => (id, s"user_${id}",
 s"user_${id}@databricks.com", states(rnd.nextInt(5))))
 .toDF("uid", "login", "email", "user_state")

usersDF: org.apache.spark.sql.DataFrame = [uid: int, login: string ... 2 more fields]


In [14]:
val ordersDF = (0 to 1000000)
 .map(r => (r, r, rnd.nextInt(10000), 10 * r* 0.2d,
 states(rnd.nextInt(5)), items(rnd.nextInt(5))))
 .toDF("transaction_id", "quantity", "users_id", "amount", "state", "items")

ordersDF: org.apache.spark.sql.DataFrame = [transaction_id: int, quantity: int ... 4 more fields]


In [15]:
val usersOrdersDF = ordersDF.join(usersDF, $"users_id" === $"uid")

usersOrdersDF: org.apache.spark.sql.DataFrame = [transaction_id: int, quantity: int ... 8 more fields]


In [24]:
usersOrdersDF.show(2, false)

java.lang.OutOfMemoryError:  Java heap space

Examining our final execution plan 

In [19]:
// usersOrderDF.explain()

In [21]:
usersDF.show(10, false)

java.lang.OutOfMemoryError:  Java heap space